<a href="https://colab.research.google.com/github/ElviraKonovalov/soen471-bigData/blob/main/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
file = 'Motor Vehicle Collisions with KSI Data.csv'
df=  pd.read_csv(file, sep=',');

columns_to_drop = ['_id', 'ACCNUM', 'TIME', 'OFFSET', 'WARDNUM', 'DIVISION', 'ACCLOC', 'FATAL_NO', 'PEDTYPE', 'PEDACT', 'PEDCOND', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'POLICE_DIVISION', 'ObjectId', 'geometry']
df = df.drop(columns=columns_to_drop)

/tmp/ipykernel_20920/401754701.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=  pd.read_csv(file, sep=',');


Features 1 - 8

Features 9 - 18

In [23]:
df = df.drop(columns = ['DRIVCOND'])


df.EMERG_VEH.fillna(0, inplace=True)
df.EMERG_VEH = df.EMERG_VEH.replace(['Yes'], 1, inplace=True)


df = df.drop(columns = ['IMPACTYPE'])


df.INITDIR.fillna("Unknown", inplace = True)


df.INVAGE = df.INVAGE.replace(['unknown'], 'Unknown') #All of the unknowns in this dataset are upper case except the ones in this collumn so it was bugging me.

#df = df.drop(columns = ['INVAGE'])


df.INVTYPE.fillna('Unknown', inplace=True)

#Removing rows where they weren't hit, as they obviously wouldn't ever have any injuries.

rows = df.INVTYPE.unique()

#['Cyclist', 'Cyclist Passenger', 'Motorcycle Passenger', 'Passenger','In-Line Skater', 'Trailer Owner', 'Other Property Owner', 'Vehicle Owner', 'Other', 'Wheelchair', 'Pedestrian', 'Unknown', 'Witness', 'Pedestrian - Not Hit', 'Driver - Not Hit']

#Remove rows I don't like
for row in rows:
  if 'Driver' not in row or 'Not Hit' in row:
    df = df[(df.INVTYPE != row)] 

#Replacing not necesarry.
#df.INVTYPE = df.INVTYPE.replace(['Motorcycle Driver', 'Moped Driver', 'Truck Driver'], 'Driver')

#Dropping the column now that I've filtered rows.
df = df.drop(columns = ['INVTYPE'])

Features 18 - 26

In [24]:
columns_to_drop = ['NEIGHBOURHOOD']
df = df.drop(columns=columns_to_drop)


#LIGHT
df.LIGHT.unique()

# Dropping all rows that have "Other" as value

df = df.drop(df[df.LIGHT == 'Other'].index)

df = df.drop(df[df.LIGHT.isnull()].index)
df.LIGHT.unique()


#LOCCOORD
df.LOCCOORD.unique()
df.LOCCOORD.isnull().sum() #105


#PEDESTRIAN
#replace every "NaN" to False and "Yes" to True

df.PEDESTRIAN.unique()
df['PEDESTRIAN'] = df['PEDESTRIAN'].replace('Yes',1, inplace=True)
df['PEDESTRIAN'] = df['PEDESTRIAN'].fillna(0)


#PASSENGER
#replace every "NaN" to False and "Yes" to True

df.PASSENGER.unique()
df['PASSENGER'] = df['PASSENGER'].replace('Yes',1, inplace=True)
df['PASSENGER'] = df['PASSENGER'].fillna(0)


#MOTORCYCLE
#replace every "NaN" to False and "Yes" to True

df.MOTORCYCLE.unique()
df['MOTORCYCLE'] = df['MOTORCYCLE'].replace('Yes',1, inplace=True)
df['MOTORCYCLE'] = df['MOTORCYCLE'].fillna(0)


df.RDSFCOND.unique()
df.RDSFCOND.isnull().sum() # 23


#REDLIGHT
#replace every "NaN" to False and "Yes" to True

df.REDLIGHT.unique()
df['REDLIGHT'] = df['REDLIGHT'].replace('Yes', 1, inplace=True)
df['REDLIGHT'] = df['REDLIGHT'].fillna(0)

Features 27 - 37

In [25]:
'''
"Every street owned by the City of Toronto has been given one of five classifications (with the exception of public laneways)."
classes: Local road, Collector road, Minor arterial road, Major arterical road and Expressway
https://www.toronto.ca/services-payments/streets-parking-transportation/traffic-management/road-classification-system/about-the-road-classification-system/
'''
# replace nan values with most frequent value in the column
df['ROAD_CLASS'] = df['ROAD_CLASS'].fillna(df['ROAD_CLASS'].value_counts().index[0])
# drop rows where ROAD_CLASS is pending
df.drop(df.index[df['ROAD_CLASS'] == 'Pending'], inplace=True)
# replacing Expressway Ramp (4 rows) with Expressway
df['ROAD_CLASS'] = df['ROAD_CLASS'].replace(['Expressway Ramp'],'Expressway')
# Major Arterial Ramp (1 row) with Major Arterial
df['ROAD_CLASS'] = df['ROAD_CLASS'].replace(['Major Arterial Ramp'],'Major Arterial')
df['ROAD_CLASS'] = df['ROAD_CLASS'].astype('category').cat.codes # label encoding


# replacing nan with 0 ('No') and 'Yes' with 1
df['SPEEDING'] = df['SPEEDING'].fillna(0)
df['SPEEDING'] = df['SPEEDING'].replace(['Yes'], 1)


# drop street names
df = df.drop(columns=['STREET1', 'STREET2'])


# group all into no control, passive control or active control
df['TRAFFCTL'] = df['TRAFFCTL'].replace(['Traffic Signal', 'Stop Sign', 'Pedestrian Crossover', 'Yield Sign', 'Streetcar (Stop for)', 'Traffic Gate'], 'Passive Control')
df['TRAFFCTL'] = df['TRAFFCTL'].replace(['Police Control', 'School Guard', 'Traffic Controller'], 'Active Control')


# replace nan with 0 ('No') and 'Yes' with 1
df['TRSN_CITY_VEH'] = df['TRSN_CITY_VEH'].fillna(0)
df.TRSN_CITY_VEH.replace(('Yes'), (1), inplace=True)


# replace nan with 0 ('No') and 'Yes' with 1
df['TRUCK'] = df['TRUCK'].fillna(0)
df.TRUCK.replace(('Yes'), (1), inplace=True)


df = df.drop(columns='VEHTYPE') # drop


# label encoding
df['VISIBILITY'] = df['VISIBILITY'].astype('category').cat.codes


df = df.drop(columns='YEAR') # drop year

In [26]:
df

,ACCLASS,AG_DRIV,ALCOHOL,AUTOMOBILE,CYCLIST,DATE,DISABILITY,DISTRICT,DRIVACT,EMERG_VEH,...,PASSENGER,PEDESTRIAN,RDSFCOND,REDLIGHT,ROAD_CLASS,SPEEDING,TRAFFCTL,TRSN_CITY_VEH,TRUCK,VISIBILITY
0,Fatal,Yes,NaN,Yes,NaN,2006-03-11T00:00:00,NaN,Toronto and East York,Failed to Yield Right of Way,None,...,0,0,Dry,0,4,0,Passive Control,0,0,0
2,Fatal,Yes,NaN,Yes,NaN,2006-03-11T00:00:00,NaN,Scarborough,Disobeyed Traffic Control,None,...,0,0,Dry,0,4,0,Passive Control,0,0,0
3,Fatal,Yes,NaN,Yes,NaN,2006-03-11T00:00:00,NaN,Scarborough,Driving Properly,None,...,0,0,Dry,0,4,0,Passive Control,0,0,0
4,Fatal,NaN,Yes,Yes,NaN,2006-03-12T00:00:00,NaN,Scarborough,Other,None,...,0,0,Dry,0,4,0,No Control,0,0,0
7,Non-Fatal Injury,Yes,Yes,Yes,NaN,2006-03-12T00:00:00,NaN,Toronto and East York,Failed to Yield Right of Way,None,...,0,0,Dry,0,4,0,Passive Control,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16853,Non-Fatal Injury,Yes,NaN,Yes,NaN,2020-08-29T00:00:00,NaN,North York,Driving Properly,None,...,0,0,Dry,0,4,0,Passive Control,0,0,0
16854,Non-Fatal Injury,Yes,NaN,Yes,Yes,2020-08-30T00:00:00,NaN,NaN,Improper Passing,None,...,0,0,Dry,0,4,0,No Control,0,0,0
16856,Non-Fatal Injury,Yes,NaN,Yes,NaN,2020-09-01T00:00:00,NaN,Scarborough,Failed to Yield Right of Way,None,...,0,0,Dry,0,4,0,Passive Control,0,0,0
16857,Non-Fatal Injury,Yes,NaN,Yes,NaN,2020-09-01T00:00:00,NaN,Scarborough,Driving Properly,None,...,0,0,Dry,0,4,0,Passive Control,0,0,0


In [27]:
df.to_csv("clean_data.csv")